In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

# 저장된 피처 튜닝 데이터 불러오기
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/악성 URL/DATA/train_FE')


In [5]:
train_df = train.loc[:,['URL', 'label', 'length', 'subdomain_count', 'special_char_count',
       'number_of_meaning_words', 'tld_malicious', 'path_depth',
       'max_numeric_sequence','https', 'blacklist_word_count',
       'digit_count', 'suspicious_keyword_flag']]



In [6]:
X,y = train_df.drop(columns=['URL','label']),train_df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],  # 트리 개수
    'criterion': ['gini'],  # 데이터 분할 기준
    'max_depth': [20, 30],  # 트리 최대 깊이 (과적합 방지)
    'min_samples_split': [50, 100, 200],  # 노드 분할 최소 샘플 수
    'min_samples_leaf': [25, 50, 100],  # 리프 노드의 최소 샘플 수
    'max_features': ['sqrt', 'log2']  # 각 노드에서 사용할 특성 수
}

grid_search = GridSearchCV(
    RandomForestClassifier(),
    param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train[:100000], y_train[:100000])

print('최적의 파라미터 조합:', grid_search.best_params_)


Fitting 5 folds for each of 72 candidates, totalling 360 fits
최적의 파라미터 조합: {'criterion': 'gini', 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 25, 'min_samples_split': 50, 'n_estimators': 200}


In [10]:
from sklearn.metrics import roc_auc_score

# 최적의 모델 가져오기
Decision_model = grid_search.best_estimator_

# 테스트 데이터에 대한 예측 확률
y_pred_proba = Decision_model.predict_proba(X_test)[:, 1]

# ROC AUC 점수 계산
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'ROC AUC Score: {roc_auc}')



ROC AUC Score: 0.9247168510896445
